In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow
import sklearn

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

In [ ]:
# read dataset
df = pd.read_csv('/kaggle/input/titanic/train.csv')
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# 결측치 확인
df.isnull().sum().sort_values(ascending=False)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Fare             0
dtype: int64

#### 불필요한 컬럼을 제외하는 작업

- Cabin 은 결측치가 너무 많아서 예측 모델에서 제외합니다.
- Name, Ticket  은 생존률에 크게 영향을 주지 않을 것이기에 제외힙니다.

In [ ]:
# Cabin 은 결측치가 너무 많아서 예측 모델에서 제외합니다.
df = df.drop(columns=['Cabin', 'Name', 'Ticket'], axis=1)
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Fare', 'Embarked'],
      dtype='object')

In [ ]:
df = pd.get_dummies(df) # # 카테고리형 변수를 0, 1로 이뤄진 변수로 바꿉니다.
df['Age'] = df['Age'].fillna(df['Age'].mean()) # Age 의 결측치를 일단 교재에 있는대로 평균값으로 채워넣었습니다.
df.isnull().sum().sort_values(ascending=False)

PassengerId    0
Survived       0
Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
dtype: int64

In [ ]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare',
       'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [ ]:
# 중요도를 체크해봅니다.
df_corr = df.corr()
df_corr_sort = df_corr.sort_values('Survived', ascending=False)
df_corr_sort['Survived'].head()

Survived      1.000000
Sex_female    0.543351
Fare          0.257307
Embarked_C    0.168240
Parch         0.081629
Name: Survived, dtype: float64

생존률에 큰 영향을 주는 것은 성별 `Sex`, 요금 `Fare`, 승선한 위치 `Embarked` 순이었습니다.

일단 `PassangerId` 를 제외한 모든 컬럼을 모델 옵션으로 설정하기로 했습니다.

In [ ]:
# 테스트를 위한 데이터를 설정합니다.
cols_train = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S']
X_train_pre = df[cols_train]
y = df['Survived'].values
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X_train_pre, y, test_size=0.2)

In [ ]:
# 모델 설정
model = Sequential()
model.add(Dense(20, input_dim=10, activation='relu', name='Dense_1'))
model.add(Dense(10, activation='relu', name='Dense_2'))
model.add(Dense(1, activation='sigmoid', name='Dense_3'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Dense_1 (Dense)             (None, 20)                220       
                                                                 
 Dense_2 (Dense)             (None, 10)                210       
                                                                 
 Dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 441
Trainable params: 441
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train, validation_split=0.25, epochs=100)

Epoch 1/100
17/17 [==============================] - 1s 18ms/step - loss: 1.0095 - accuracy: 0.3652 - val_loss: 0.8427 - val_accuracy: 0.2921
Epoch 2/100
17/17 [==============================] - 0s 4ms/step - loss: 0.7794 - accuracy: 0.3933 - val_loss: 0.7242 - val_accuracy: 0.5056
Epoch 3/100
17/17 [==============================] - 0s 4ms/step - loss: 0.7110 - accuracy: 0.5393 - val_loss: 0.6674 - val_accuracy: 0.5843
Epoch 4/100
17/17 [==============================] - 0s 5ms/step - loss: 0.6711 - accuracy: 0.5805 - val_loss: 0.6137 - val_accuracy: 0.6573
Epoch 5/100
17/17 [==============================] - 0s 4ms/step - loss: 0.6426 - accuracy: 0.5993 - val_loss: 0.5734 - val_accuracy: 0.6742
Epoch 6/100
17/17 [==============================] - 0s 4ms/step - loss: 0.6321 - accuracy: 0.5974 - val_loss: 0.5623 - val_accuracy: 0.6798
Epoch 7/100
17/17 [==============================] - 0s 4ms/step - loss: 0.6296 - accuracy: 0.5993 - val_loss: 0.5660 - val_accuracy: 0.6685
Epoch 8/100


In [ ]:
model.evaluate(X_test, y_test)
# 정확도는 82%가 나왔습니다.

6/6 [==============================] - 0s 2ms/step - loss: 0.4316 - accuracy: 0.8212


[0.43157798051834106, 0.8212290406227112]

In [ ]:
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')
test_df = test_df.drop(columns=['Cabin', 'Name', 'Ticket'], axis=1)
test_df = pd.get_dummies(test_df)

In [ ]:
prediction = model.predict(test_df[cols_train])

14/14 [==============================] - 0s 2ms/step
